In [62]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [63]:
data = pd.read_excel('politeness_finalset_data.xlsx')

In [64]:
data.fillna(0, inplace=True)

In [65]:
X_train = data.comment.tolist()
y_train=data['final_agreed_rating'].values
y_train=y_train.tolist()

In [50]:
class_names = ['polite', 'impolite', 'neutral']
print(data.final_agreed_rating.value_counts())


neutral     322
polite      220
impolite     46
0             1
Name: final_agreed_rating, dtype: int64


In [66]:
encoding = {
    'impolite': 'impolite',
    'neutral': 'impolite',
    'polite': 'polite',
    0: 'polite',
}

In [67]:
y_train = [encoding[x] for x in y_train]


In [68]:
#print(y_train)

In [69]:
class_names = ['polite', 'impolite']
#print(y_train.value_counts())

In [70]:
encoding = {
    'impolite': 0,
    'polite': 1,
    0: 1,
}

In [71]:
y_train = [encoding[x] for x in y_train]


In [72]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train_new, y_test= train_test_split(X_train, y_train, test_size=0.1, random_state=125)


In [73]:
x_train, x_val, y_train, y_val= train_test_split(x_train,  y_train_new, test_size=0.21, random_state=125)

In [74]:
(x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=['polite','impolite'],
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=3500)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [75]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)


Is Multi-Label? False
maxlen is 350
done.


In [76]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [77]:
import tensorflow as tf
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)

In [78]:
learner.fit_onecycle(2e-5, 10, callbacks=[callbacks])




begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
70/70 [==============================] - 488s 7s/step - loss: 0.6224 - accuracy: 0.6364 - val_loss: 0.6520 - val_accuracy: 0.5424
Epoch 2/10
70/70 [==============================] - 461s 7s/step - loss: 0.4879 - accuracy: 0.7727 - val_loss: 0.6233 - val_accuracy: 0.6610
Epoch 3/10
70/70 [==============================] - 465s 7s/step - loss: 0.2973 - accuracy: 0.8852 - val_loss: 0.6591 - val_accuracy: 0.6949


In [79]:
(x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_val, y_train=y_val,
                                                                       x_test=x_val, y_test=y_val,
                                                                       
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=3500)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [80]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)


              precision    recall  f1-score   support

      polite       0.92      0.72      0.81        76
    impolite       0.60      0.86      0.70        36

    accuracy                           0.77       112
   macro avg       0.76      0.79      0.76       112
weighted avg       0.81      0.77      0.78       112



array([[55, 21],
       [ 5, 31]])